In [1]:
# Dependencies
import requests
import json
import pandas as pd
from config import api_key
from pandas.io.json import json_normalize  

## Get Scopus Author Information using Multiple Scopus Author IDs

In [2]:
# File to Load
radiation_oncology = "radiation_oncology.csv"

# Read the CSV file and store into Pandas DataFrame with the column Scopus Author ID as a string
radiation_oncology_df = pd.read_csv(radiation_oncology, encoding="utf-8",dtype={'Scopus Author ID': str})

#Change the column names to lower case with underscore for spaces
radiation_oncology_df.columns =  radiation_oncology_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
radiation_oncology_df.head()

,last_name,first_name,mi,department,netid,position,division,career_track,in_elements?,scopus_author_id,scopus_search,unnamed:_11
0,Donnelly,Eric,D,Radiation Oncology,edo615,Associate Professor,NaN,Clinician-Educator,Yes,21233377200,AU-ID(21233377200),AU-ID(21233377200) OR
1,Gentile,Michelle,S,Radiation Oncology,msg842,Assistant Professor,NaN,Clinician-Educator,Yes,56018970700,AU-ID(56018970700),AU-ID(56018970700) OR
2,Gius,David,R,Radiation Oncology,drg815,Professor,NaN,Investigator,Yes,7003610066,AU-ID(7003610066),AU-ID(7003610066) OR
3,Hayes,John,P,Radiation Oncology,jph125,Assistant Professor,NaN,Clinician-Educator,Yes,55313350000,AU-ID(55313350000),AU-ID(55313350000) OR
4,Kalapurakal,John,A,Radiation Oncology,jak801,Professor,NaN,Clinician-Educator,Yes,7003993738,AU-ID(7003993738),AU-ID(7003993738) OR


In [3]:
#List the column names in the dataframe
# radiation_oncology_df.columns

In [4]:
#Check the data type in the dataframe columns called scopus_author_id and scopus_search
# radiation_oncology_df.scopus_author_id.dtype
# radiation_oncology_df.scopus_search.dtype

In [5]:
#Save the column called scopus_author_id to a list called Author_ID_List
author_ID_List = radiation_oncology_df['scopus_author_id'].tolist()
print(author_ID_List)

['21233377200', '56018970700', '7003610066', '55313350000', '7003993738', '24448583300', '7102661470', '57207807061', '56443683800', '6701754514', '36143455500', '24336584500', '7005165328', '453']


In [6]:
#Clean the Author_ID_List to remove nan
cleaned_Author_ID_List = [x for x in author_ID_List if str(x) != 'nan']
print(cleaned_Author_ID_List)

['21233377200', '56018970700', '7003610066', '55313350000', '7003993738', '24448583300', '7102661470', '57207807061', '56443683800', '6701754514', '36143455500', '24336584500', '7005165328', '453']


In [7]:
#Add the necessary syntax to the cleaned_Author_ID_List
scopus_Mulitple_AuthorID_Query = []
for x in cleaned_Author_ID_List:
    authorID_string = "".join(("AU-ID(", x,")"))
    scopus_Mulitple_AuthorID_Query.append(authorID_string)
    
print(scopus_Mulitple_AuthorID_Query)

['AU-ID(21233377200)', 'AU-ID(56018970700)', 'AU-ID(7003610066)', 'AU-ID(55313350000)', 'AU-ID(7003993738)', 'AU-ID(24448583300)', 'AU-ID(7102661470)', 'AU-ID(57207807061)', 'AU-ID(56443683800)', 'AU-ID(6701754514)', 'AU-ID(36143455500)', 'AU-ID(24336584500)', 'AU-ID(7005165328)', 'AU-ID(453)']


In [8]:
# scopus_Search_List = radiation_oncology_df['scopus_search'].tolist()
# print(scopus_Search_List)

In [9]:
# cleaned_Scopus_Search_List = [x for x in scopus_Search_List if str(x) != 'nan']
# print(cleaned_Scopus_Search_List)

In [14]:
#https://dev.elsevier.com/guides/ScopusSearchViews.htm
def get_scopus_articles(scopus_Mulitple_AuthorID_Query):
    for authid in scopus_Mulitple_AuthorID_Query:
        url = "http://api.elsevier.com/content/search/scopus?"
        fieldList = ["dc:identifier", "eid", "dc:title","prism:aggregationType", "subtype", "citedby-count",
                     "prism:publicationName","prism:volume","prism:issueIdentifier", "prism:pageRange", 
                     "prism:coverDate", "prism:doi","pubmed-id", "authid", "authname"]
                    
        headers = {
             "X-ELS-APIKey": api_key,
             'Accept':'application/json'
        }
        parameters = {
            "query": authid,
            "field": ",".join(fieldList),
            "date": "2002-2003"
        }
        #Make the API request
        multiple_author_response = requests.get(url, headers=headers, params=parameters)
        print(multiple_author_response.url)
        print(multiple_author_response.status_code)
        
        #Format the response using json.dumps
        response_multiple_author_search = multiple_author_response.json()
        response_multiple_author_search_cleaned = json.dumps(response_multiple_author_search, indent=4, sort_keys=True)
        print(response_multiple_author_search_cleaned)
        
    return(response_multiple_author_search_cleaned)
get_scopus_articles(scopus_Mulitple_AuthorID_Query)

https://api.elsevier.com/content/search/scopus?query=AU-ID%2821233377200%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003
200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "error": "Result set was empty"
            }
        ],
        "link": [
            {
                "@_fa": "true",
                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=25&query=AU-ID%2821233377200%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003",
                "@ref": "self",
             

https://api.elsevier.com/content/search/scopus?query=AU-ID%2855313350000%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003
200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "error": "Result set was empty"
            }
        ],
        "link": [
            {
                "@_fa": "true",
                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=25&query=AU-ID%2855313350000%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003",
                "@ref": "self",
             

https://api.elsevier.com/content/search/scopus?query=AU-ID%2824448583300%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003
200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "error": "Result set was empty"
            }
        ],
        "link": [
            {
                "@_fa": "true",
                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=25&query=AU-ID%2824448583300%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003",
                "@ref": "self",
             

https://api.elsevier.com/content/search/scopus?query=AU-ID%2857207807061%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003
200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "author": [
                    {
                        "@_fa": "true",
                        "authid": "7102661470",
                        "authname": "Mittal B.",
                        "author-url": "https://api.elsevier.com/content/author/author_id/7102661470"
                    },
                    {
                        "@_fa": "true",
                        "authid": "7003814312",
                        "authname": "Pauloski B.",
                        "author-url": "https://api.elsevier.com/content/author/author_id/70038

https://api.elsevier.com/content/search/scopus?query=AU-ID%2856443683800%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003
200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "error": "Result set was empty"
            }
        ],
        "link": [
            {
                "@_fa": "true",
                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=25&query=AU-ID%2856443683800%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003",
                "@ref": "self",
             

https://api.elsevier.com/content/search/scopus?query=AU-ID%28453%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003
200
{
    "search-results": {
        "entry": [
            {
                "@_fa": "true",
                "error": "Result set was empty"
            }
        ],
        "link": [
            {
                "@_fa": "true",
                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=25&query=AU-ID%28453%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003",
                "@ref": "self",
                "@type": "app

'{\n    "search-results": {\n        "entry": [\n            {\n                "@_fa": "true",\n                "error": "Result set was empty"\n            }\n        ],\n        "link": [\n            {\n                "@_fa": "true",\n                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=25&query=AU-ID%28453%29&field=dc%3Aidentifier%2Ceid%2Cdc%3Atitle%2Cprism%3AaggregationType%2Csubtype%2Ccitedby-count%2Cprism%3ApublicationName%2Cprism%3Avolume%2Cprism%3AissueIdentifier%2Cprism%3ApageRange%2Cprism%3AcoverDate%2Cprism%3Adoi%2Cpubmed-id%2Cauthid%2Cauthname&date=2002-2003",\n                "@ref": "self",\n                "@type": "application/json"\n            }\n        ],\n        "opensearch:Query": {\n            "@role": "request",\n            "@searchTerms": "AU-ID(453)",\n            "@startPage": "0"\n        },\n        "opensearch:itemsPerPage": "0",\n        "opensearch:startIndex": "0",\n        "opensearch:totalResults": "0"\n    }\n}